# Introduction

### Configuration

In [ ]:
# The cache directories:
cache_path       = '/data/crcns2021'
image_cache_path = f'{cache_path}/annot-images'
v123_cache_path  = f'{cache_path}/annot-v123'
csulc_cache_path = f'{cache_path}/annot-csulc'
trace_save_path = '/home/nben/code/hcp-annot-vc_data/save'

### Dependencies

In [ ]:
import os, sys, pimms, pandas, warnings, urllib, datetime
import numpy as np
import scipy as sp
import scipy.sparse as sps
import nibabel as nib
import neuropythy as ny

import matplotlib as mpl
import matplotlib.pyplot as plt
import ipyvolume as ipv
import torch, json

In [ ]:
%matplotlib inline

In [ ]:
# Additional matplotlib preferences:
font_data = {'family':'sans-serif',
             'sans-serif':['HelveticaNeue', 'Helvetica', 'Arial'],
             'size': 10,
             'weight': 'light'}
mpl.rc('font',**font_data)
# we want relatively high-res images, especially when saving to disk.
mpl.rcParams['figure.dpi'] = 72*2
mpl.rcParams['savefig.dpi'] = 72*4

### Import `hcpannot`

In [ ]:
# Make sure we are in the right directory for this!
cwd = os.getcwd()
if cwd.endswith('/work'):
    os.chdir('..')
    cwd = os.getcwd()

# Now we can import the hcp-annot-v123 library.
import hcpannot

# Note the cache path we want to use outside the docker container;
# normally this gets set by the Docker startup, so here outside
# the docker container we set it manually.
hcpannot.interface.default_load_path = cache_path
# We have to update some data after setting this.

# We also want to grab a few variables from hcpannot:
from hcpannot.interface import default_imshape as roi_image_shape
roi_image_shape = roi_image_shape[0] // 2
from hcpannot.interface import imgrid_to_flatmap
from hcpannot.interface import flatmap_to_imgrid
# The analysis plan (for processing contours):
from hcpannot import vc_plan
# The functions for loading and saving contours.
from hcpannot import (save_contours, load_contours)

# Subject lists. These are defined in the analysis subpackage of
# the hcpannot library, and subject_list_<x> is the <x>th list of
# subject IDs that we assigned.
from hcpannot import (subject_ids, subject_list_1,
                      subject_list_2, subject_list_3)
sids = np.array(hcpannot.subject_ids)

# We also want to import the IO functions.
from hcpannot.io import (save_traces, export_traces, load_traces, export_paths,
                         load_paths, export_means, calc_surface_areas)

# The mean rater's name ('mean') is also defined in hcpannot.
from hcpannot.analysis import meanrater

### Utility Functions

#### Plotting Functions

In [ ]:
raw_colors = {
    'hV4_outer': (0.5, 0,   0),
    'hV4_VO1':   (0,   0.3, 0),
    'VO_outer':  (0,   0.4, 0.6),
    'VO1_VO2':   (0,   0,   0.5)}
preproc_colors = {
    'hV4_outer': (0.7, 0,   0),
    'hV4_VO1':   (0,   0.5, 0),
    'VO_outer':  (0,   0.6, 0.8),
    'VO1_VO2':   (0,   0,   0.7),
    'V3_ventral':(0.7, 0,   0.7),
    'outer':     (0.7, 0.7, 0, 0)}
ext_colors = {
    'hV4_outer': (1,   0,   0),
    'hV4_VO1':   (0,   0.8, 0),
    'VO_outer':  (0,   0.9, 1),
    'VO1_VO2':   (0,   0,   1),
    'V3_ventral':(0.8, 0,   0.8),
    'outer':     (0.8, 0.8, 0.8, 1)}
boundary_colors = {
    'hV4': (1, 0.5, 0.5),
    'VO1': (0.5, 1, 0.5),
    'VO2': (0.5, 0.5, 1)}

def plot_contours(dat, raw=None, ext=None, preproc=None,
                  contours=None, boundaries=None,
                  figsize=(2,2), dpi=(72*5), axes=None, 
                  flatmap=True, lw=1, color='prf_polar_angle',  
                  mask=('prf_variance_explained', 0.05, 1)):
    # Make the figure.
    if axes is None:
        (fig,ax) = plt.subplots(1,1, figsize=figsize, dpi=dpi)
        fig.subplots_adjust(0,0,1,1,0,0)
    else:
        ax = axes
        fig = ax.get_figure()
    # Plot the flatmap.
    if flatmap:
        fmap = dat['flatmap']
        ny.cortex_plot(fmap, color=color, mask=mask, axes=ax)
    # Plot the requested lines:
    if raw is not None:
        if raw is True: raw = raw_colors
        for (k,v) in dat['raw_contours'].items():
            c = raw.get(k, 'w')
            ax.plot(v[0], v[1], '-', color=c, lw=lw)
    if preproc is not None:
        if preproc is True: preproc = preproc_colors
        for (k,v) in dat['preproc_contours'].items():
            c = preproc.get(k, 'w')
            ax.plot(v[0], v[1], '-', color=c, lw=lw)
    if ext is not None:
        if ext is True: ext = ext_colors
        for (k,v) in dat['ext_contours'].items():
            c = ext.get(k, 'w')
            ax.plot(v[0], v[1], '-', color=c, lw=lw)
    if contours is not None:
        if contours is True: contours = ext_colors
        for (k,v) in dat['contours'].items():
            c = contours.get(k, 'w')
            ax.plot(v[0], v[1], '-', color=c, lw=lw)
    if boundaries is not None:
        if boundaries is True: boundaries = boundary_colors
        for (k,v) in dat['boundaries'].items():
            c = boundaries.get(k, 'w')
            x = np.concatenate([v[0], [v[0][0]]])
            y = np.concatenate([v[1], [v[1][0]]])
            ax.plot(x, y, '-', color=c, lw=lw)
    ax.axis('off')
    return fig

Here's an example code-block using the above to plot a set of boundaries along with the V1-V3 contours.

```python
dat = vc_plan(rater='BrendaQiu',
              sid=111312,
              hemisphere='lh',
              save_path=trace_save_path)
# Make the flatmap plot and the boundaries.
fig = plot_contours(dat, boundaries=True)
# Extract the pyplot axes that were used.
ax = fig.axes[0]
# Grab the subject data, which includes the V1-V3 contours.
sdat = hcpannot.interface.subject_data[(dat['sid'],dat['hemisphere'])]
# And plot all of these contours:
for (x,y) in sdat['v123'].values():
    ax.plot(x, y, 'w-', lw=0.25)
```

#### Export/Multiprocessing Functions

In [ ]:
# This function gets used below to pass tuples of arguments to functions through
# the multiprocessing pool interface.
def tupcall(fn, tup=None,
            onfail=None,
            onokay=None,
            retryon=None):
    """Passes a tuple of arguments to a function.

    `tupcall(fn, tuple)` calls `fn(*tuple)` and returns the result. If the final
    element of `tuple` is a dictionary, then the return value is instead
    `fn(*tuple[:-1], **tuple[-1])`. An empty dictionary may be passed as the
    final element of `tuple`.
    
    `tupcall(fn)` returns a function `g` such that `g(tuple)` is equivalent to
    `tupcall(fn, tuple)`.
    
    The optional argument `onfail` can be set to a backup function that is 
    called if an exception is raised. In such a case, the return value of the
    `tupcall` function is `onfail(raised_exception, tuple)`.
    
    The optional argument `onokay` can be set to a function that is 
    called if no exception is raised. In such a case, the return value of the
    `tupcall` function is `onokay(raised_error, tuple)`.

    The optional argument `retryon` can be set to an exception type or a tuple
    of exception types; if one of these exceptions is raised during execution,
    then the function is retried once. Alternately, `retryon` may be a dict
    whose keys are exception types or tuples of exception types and whose values
    are functions that determine whether or not to retry the call. These
    functions are called as `do_retry = fn(raised_error, tuple)`, and `do_retry`
    must be a boolean result.
    """
    if tup is None:
        from functools import partial
        return partial(tupcall, fn,
                       onfail=onfail, onokay=onokay, retryon=retryon)
    elif not isinstance(tup, (tuple, list)):
        raise ValueError("tupcall requires a tuple or list")
    elif len(tup) == 0:
        fin = {}
        arg = tup
    else:
        fin = tup[-1]
        if isinstance(fin, dict):
            arg = tup[:-1]
        else:
            fin = {}
            arg = tup
    if retryon is None and onfail is None:
        res = fn(*arg, **fin)
    else:
        while True:
            try:
                res = fn(*arg, **fin)
                break
            except Exception as e:
                if retryon is not None:
                    if isinstance(retryon, dict):
                        do_retry = False
                        for (k,fn) in retryon.items():
                            if isinstance(e, k):
                                do_retry = fn(e, tup)
                                if do_retry: break
                        if do_retry: continue
                    elif isinstance(e, retryon):
                        # We only retry once.
                        retryon = None
                        continue
                # If we reach here, we aren't retrying.
                if onfail:
                    return onfail(e, tup)
                else:
                    raise                    
    if onokay is not None:
        return onokay(res, tup)
    else:
        return res

In [ ]:
def okaystr(res, tup):
    """Returns a string summary of an okay result for use with `tupcall`.
    
    `okaystr(r, tuple)` returns a string `f"OKAY: {tuple}"`.
    """
    return f"OKAY: {tup}"
def failstr(err, tup):
    """Returns a string summary of a failed result for use with `tupcall`.
    
    `failstr(err, tuple)` returns a string `f"FAIL: {tuple} {err}"`.
    """
    return f"FAIL: {tup} {err}"
def retry_sleep(err=None, tup=None, duration=Ellipsis):
    """Sleeps for 5 seconds then returns `True`, for use with `tupcall`.

    `retry_sleep(error, tuple)` sleeps for 5 seconds then returns `True`.
    
    `retry_sleep(error, tuple, duration=dur)` sleeps for `dur` seconds then
    returns `True`.
    
    `retry_sleep(dur)` returns a function equivalent to
    `lambda err, tup: retry_sleep(err, tup, duration=dur)`.
    
    This function is intended for use with `tupcall`'s `retryon` option, for
    example, `tupcall(fn, tup, retryon={HTTPError: retry_sleep(5)})`.
    """
    import time
    if err is None and tup is None:
        dur = duration
    elif tup is None and duration is Ellipsis:
        dur = err
    elif tup is None:
        raise ValueError(f"invalid arguments to retry_sleep:"
                         f" ({err},{tup},{duration})")
    else:
        dur = 5 if duration is Ellipsis else duration
        time.sleep(dur)
        return True
    # If we make it here, we need to return a partial function.
    return lambda err,tup: retry_sleep(err, tup, duration=dur)

In [ ]:
# We actually want to retry on HTTPError, which occurs when we overload OSF with
# download requests.
def mprun(fn, jobs,
          nproc=None,
          print=print,
          onokay=okaystr,
          onfail=failstr,
          retryon={urllib.error.HTTPError: retry_sleep}):
    """Runs a function across many processes via the `multiprocessing` package.
    
    `mprun(fn, joblist)` runs the given function across as many processes as
    there are CPUs for each job in `joblist`. The jobs should be tuples that
    can be executed via `tupcall(fn, job)`.
    
    The optional arguments `onfail` and `onokay` are passed through to the
    `tupcall` function. Additionally, the option `nproc` can be set to the
    number of processes that should be used; the default of `None` indicates
    that the number of processes should match the number of CPUs. Finally, the
    option `print` may be set to a print function that is used to log the
    progress of the run. If `None` is given, then no printing is done;
    otherwise, every 10% of the total set of jobs complete produces a progress
    message.
    """
    import multiprocessing as mp
    # Process the arguments.
    njobs = len(jobs)
    try:
        fnname = fn.__name__
    except Exception:
        fnname = str(fn)
    if nproc is None:
        nproc = mp.cpu_count()
    callfn = tupcall(fn, onfail=onfail, onokay=onokay, retryon=retryon)
    # Start the jobs!
    print(f"Beginning {njobs} jobs with tag '{tag}'...")
    donecount = 0
    res = []
    for ii in range(0, njobs, nproc):
        jj = min(ii + nproc, njobs)
        nn = jj - ii
        with mp.Pool(nn) as pool:
            r = pool.map(callfn, jobs[ii:jj])
        for rr in r:
            res.append(rr)
        if donecount * 10 // njobs < (donecount + nn) * 10 // njobs:
            print(" - %4d / %4d (%3d%%)" % (jj, njobs, int(100*jj/njobs)))
        donecount += nn
    return res

In [ ]:
def mpstep(fn, jobs, tag,
           overwrite=False, nproc=None, save_path=trace_save_path,
           onokay=okaystr, onfail=failstr, print=print,
           retryon={urllib.error.HTTPError: retry_sleep}):
    """Runs one multiprocessing step in the contour process/export workflow.
    
    `mpstep(fn, jobs, tag)` is designed to be run with the `hcpannot.io`
    functions for exporting processed data about the contours:
      * `export_traces`
      * `export_paths`
      * `export_means`
    Each of these functions must be multiprocessed across many combinations of
    raters, subjects, and hemispheres. These arguments must be listed in `jobs`.
    The `tag` is used to name the logfile that is exported on success.
    
    Parameters
    ----------
    fn : function
        The function that is to be multiprocessed across all jobs.
    jobs : list of tuples
        A list of arguments to `fn`; see `makejobs` and `tupcall`.
    tag : str
        A tag name used to identify the logfile, which is placed in the
        `save_path` directory.
    overwrite : boolean, optional
        If overwrite is `False` and the logfile already exists, then it is read
        in and returned instead of rerunning the jobs. The default is `False`.
    nproc : int or None, optional
        The number of processes to multiplex across. If this is `None`, then the
        number of CPUs is used. The default is `None`.
    save_path : directory name, optional
        The directory from which to load the contour data and into which to
        write the logfile. The default is the global variable `trace_save_path`.
    
    Returns
    -------
    list of str
        A list with one entry per job that is the `okaystr` or `failstr`
        representation of that job's return value.
    """
    # We'll write out this logfile.
    #logfile = f"{tag}_{datetime.datetime.now().isoformat()}.log"
    logfile = f"proc_{tag}.log"
    logfile = os.path.join(save_path, logfile)
    # Multiprocess all the jobs.
    if overwrite or not os.path.isfile(logfile):
        proc_results = mprun(fn, jobs,
                             nproc=nproc,
                             print=print,
                             onokay=onokay,
                             onfail=onfail,
                             retryon=retryon)
        # Write out a log of these results.
        with open(logfile, "wt") as fl:
            fl.write('\n'.join(proc_results))
            fl.write('\n')
    else:
        with open(logfile, "rt") as fl:
            proc_results = fl.readlines()
    return proc_results

In [ ]:
def makejobs(*args):
    "Equivalent to `itertools.product(*args)` but always returns a list."
    from itertools import product
    return list(product(*args))

## Work

### Preprocessing the Contours into Boundaries

#### Initial Parameters

These parameters are used throughout the code below that process the contours into area boundaries and surface areas.

In [ ]:
# The raters we are processing over.
proc_raters = [
    'BrendaQiu',
    'bogengsong',
    'JiyeongHa',
    'lindazelinzhao',
    'nourahboujaber',
    'jennifertepan']

# The subject IDs we are processing over.
proc_sids = hcpannot.subject_ids

# The hemispheres we are processing over.
proc_hemis = ['lh', 'rh']

# The number of processes we want to use (None for all CPUs).
nproc = 24

# How we log information about the processing; typically just the print
# function. Note that we always write out log-files containing the results of
# processing each rater/sid/hemisphere; this is just about printing to the
# notebook/screen.
logfn = print

# If we want to skip this step whenever the logfile already exists, we can set
# the overwrite value to False. If this is True, then the export trace functions
# will always be run.
overwrite = False

#### Traces and Paths

In [ ]:
# First, we process the traces.

# Options we want to pass to the export_traces funciton.
opts = {'overwrite': overwrite}
# Make the job list.
jobs = makejobs(proc_raters, proc_sids, proc_hemis, [trace_save_path], [opts])
# Run this step in the processing.
proc_traces_results = mpstep(export_traces, jobs, "traces",
                             overwrite=overwrite,
                             nproc=nproc,
                             print=logfn)

In [ ]:
# Next, we process the paths.

# Options we want to pass to the export_paths funciton.
opts = {'overwrite': overwrite}
# Make the job list.
jobs = makejobs(proc_raters, proc_sids, proc_hemis, [trace_save_path], [opts])
# Run this step in the processing.
proc_paths_results = mpstep(export_paths, jobs, "paths",
                            overwrite=overwrite,
                            nproc=nproc,
                            print=logfn)

#### Mean Contours, Traces, and Paths

In [ ]:
# Here we create the mean contours.

# Options we want to pass to the export_means funciton.
opts = {'overwrite': overwrite}
# Make the job list.
jobs = makejobs(proc_sids, proc_hemis, [trace_save_path], [opts])
# Run this step in the processing.
proc_means_results = mpstep(export_means, jobs, "means",
                            overwrite=overwrite,
                            nproc=nproc,
                            print=logfn)

In [ ]:
# Next, we create the traces for the mean rater.

# Options we want to pass to the export_traces funciton.
opts = {'vc_contours': hcpannot.analysis.vc_contours_meanrater,
        'overwrite': overwrite}
# Make the job list.
jobs = makejobs([meanrater], proc_sids, proc_hemis, [trace_save_path], [opts])
# Run this step in the processing.
proc_meantraces_results = mpstep(export_traces, jobs, "meantraces",
                                 overwrite=overwrite,
                                 nproc=nproc,
                                 print=logfn)

In [ ]:
# Finally, we create the paths for the mean rater.

# Options we want to pass to the export_paths funciton.
opts = {'overwrite': overwrite}
# Make the job list.
jobs = makejobs([meanrater], proc_sids, proc_hemis, [trace_save_path], [opts])
# Run this step in the processing.
proc_meanpaths_results = mpstep(export_paths, jobs, "meanpaths",
                                overwrite=overwrite,
                                nproc=nproc,
                                print=logfn)

#### Surface Areas

Note that the cell below will generate a large number of warnings about poor performance right toward the end of the analysis. This is because the mean subjects all have large numbers of points in their contours, and this causes poorer performance. Fortunately, these still run in a reasonable time, so the warnings can be ignored.

In [ ]:
# For calculating surface areas, we use okay and fail functions that return an
# okaystr or failstr bur that also records a dict of data for the dataframe.
def sarea_fail(err, tup):
    msg = failstr(err, tup)
    (rater, sid, h, save_path, opts) = tup
    res = dict(
        rater=rater, sid=sid, hemisphere=h,
        hV4=np.nan, VO1=np.nan, VO2=np.nan, cortex=np.nan,
        message=msg)
    return res
def sarea_okay(res, tup):
    msg = okaystr(res, tup)
    res = dict(res, message=msg)
    return res

In [ ]:
# We save the sarea file to this location.
sarea_filename = os.path.join(trace_save_path, 'surface_areas.csv')
# We use the standard raters plus the meanrater here.
proc_sarea_raters = proc_raters + [meanrater]
# Options we want to pass to calc_surface_areas.
opts = {}
# Make the job list.
jobs = makejobs(proc_sarea_raters, proc_sids, proc_hemis,
                [trace_save_path], [opts])
# Multiprocess all the jobs.
if overwrite or not os.path.isfile(sarea_filename):
    proc_sarea_results = mprun(calc_surface_areas, jobs,
                               nproc=nproc,
                               print=logfn,
                               onokay=sarea_okay,
                               onfail=sarea_fail)
    # Convert the surface area data to a dataframe and save it.
    sarea_data = ny.to_dataframe(proc_sarea_results)
    ny.save(sarea_filename, sarea_data)
else:
    sarea_data = ny.load(sarea_filename)
    proc_sarea_results = None
proc_sarea_results = [r['message'] for (ii,r) in sarea_data.iterrows()]
sarea_data = sarea_data[['rater','sid','hemisphere','hV4','VO1','VO2','cortex']]

## Scratch

In [ ]:
sa_nans = sarea_data[np.isnan(sarea_data['hV4'])]
# Dataframe of raters and how many NaN surface areas they have.
# The total number of entries should always be 362.
raters = sa_nans['rater'].values
missing_sarea_data = ny.to_dataframe(
    [dict(rater=rater,
          nancount=np.sum(sa_nans['rater'] == rater),
          entrycount=np.sum(sarea_data['rater'] == rater))
     for rater in np.unique(raters)])
missing_sarea_data

In [ ]:
rsp = ny.data['hcp_lines'].retinotopy_sibling_pairs
rsp = {
    k: v[np.all(np.isin(v, subject_list_1), axis=1)]
    for (k,v) in rsp.items()}    

In [ ]:
while True:
    mean_twin_sids = {k: v[np.random.choice(np.arange(v.shape[0]))]
                      for (k,v) in rsp.items()
                      if k != 'nontwin_siblings'}
    mean_twin_trs = {}
    for h in ['lh','rh']:
        trs = {}
        try:
            for (k,(t1,t2)) in mean_twin_sids.items():
                trs[k] = {t1:mean_traces(t1, h), t2:mean_traces(t2, h)}
            mean_twin_trs[h] = trs
        except Exception:
            raise
    if len(mean_twin_trs) == 2: break

In [ ]:
(fig,axs) = plt.subplots(2,4, figsize=(4,2), dpi=72*8)
fig.subplots_adjust(0,0,1,1,0.1,0.1)

mask = ('prf_variance_explained', 0.025, 1)
lims = ((-60,60), (-80,40))

for ((h,trs),axrow) in zip(mean_twin_trs.items(), axs):
    mz = trs['monozygotic_twins']
    dz = trs['dizygotic_twins']
    mm = {k:v for d in (mz,dz) for (k,v) in d.items()}
    for (ax,(sid,trs)) in zip(axrow, mm.items()):
        # Get this sujbect's flatmap
        sub = ny.data['hcp_lines'].subjects[sid]
        hem = sub.hemis[h]
        fmap = hcpannot.op_flatmap(hem)
        # Plot the flatmap.
        ny.cortex_plot(fmap, color='prf_polar_angle', axes=ax,
                       mask=mask)
        sdat = hcpannot.interface.subject_data[(sid,h)]
        for (k,v) in sdat['v123'].items():
            (x,y) = v
            ax.plot(x, y, 'w-', lw=0.25)
        for (k,v) in trs.items():
            (x,y) = v.points
            ax.plot(x, y, 'k-', lw=0.5)
        ax.axis('off')
        ax.set_xlim(lims[0])
        ax.set_ylim(lims[1])

pass

In [ ]:
df_long = {k: np.concatenate([v, v, v, v])
           for k in ['rater','sid','hemisphere']
           for v in [df[k].values]}
n = len(df)
df_long['visual_area'] = np.concatenate(
    [[k]*n for k in ['hV4','VO1','VO2','cortex']])
df_long['surface_area'] = np.concatenate(
    [df[k].values for k in ['hV4','VO1','VO2','cortex']])
df_long = ny.to_dataframe(df_long)

In [ ]:
import seaborn as sns

df_plot = df_long[df_long['visual_area'] != 'cortex']
ax = sns.violinplot(x="visual_area", y="surface_area", hue="hemisphere",
                    col='rater', split=True, data=df_plot, cut=0, lw=0.5)
ax.set_ylabel(r'Surface Area [mm$^2$]')
ax.set_xlabel('Visual Area')

pass

In [ ]:
df_plot = df_long[df_long['visual_area'] != 'cortex']
sns.catplot(x="rater", y="surface_area", hue="hemisphere",
            col='visual_area', split=True, data=df_plot, cut=0,
            kind="violin")
#ax.set_ylabel(r'Surface Area [mm$^2$]')
#ax.set_xlabel('Visual Area')

pass